# Homework Final Análisis Predictivo

- Lucía Van-Halen Sáez

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
# Cargar los datos
df = pd.read_csv('H2.csv')  # Importamos el csv

### Preprocesamiento de los datos

In [3]:
# Ver si hay NA´s en nuestros datos 
missing_data = df.isnull().sum()
print(missing_data)

IsCanceled                      0
LeadTime                        0
ArrivalDateYear                 0
ArrivalDateMonth                0
ArrivalDateWeekNumber           0
ArrivalDateDayOfMonth           0
StaysInWeekendNights            0
StaysInWeekNights               0
Adults                          0
Children                        4
Babies                          0
Meal                            0
Country                        24
MarketSegment                   0
DistributionChannel             0
IsRepeatedGuest                 0
PreviousCancellations           0
PreviousBookingsNotCanceled     0
ReservedRoomType                0
AssignedRoomType                0
BookingChanges                  0
DepositType                     0
Agent                           0
Company                         0
DaysInWaitingList               0
CustomerType                    0
ADR                             0
RequiredCarParkingSpaces        0
TotalOfSpecialRequests          0
ReservationSta

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79330 entries, 0 to 79329
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   IsCanceled                   79330 non-null  int64  
 1   LeadTime                     79330 non-null  int64  
 2   ArrivalDateYear              79330 non-null  int64  
 3   ArrivalDateMonth             79330 non-null  object 
 4   ArrivalDateWeekNumber        79330 non-null  int64  
 5   ArrivalDateDayOfMonth        79330 non-null  int64  
 6   StaysInWeekendNights         79330 non-null  int64  
 7   StaysInWeekNights            79330 non-null  int64  
 8   Adults                       79330 non-null  int64  
 9   Children                     79326 non-null  float64
 10  Babies                       79330 non-null  int64  
 11  Meal                         79330 non-null  object 
 12  Country                      79306 non-null  object 
 13  MarketSegment   

In [5]:
df.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,0,6,2015,July,27,1,0,2,1,0.0,...,No Deposit,6,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-03
1,1,88,2015,July,27,1,0,4,2,0.0,...,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-07-01
2,1,65,2015,July,27,1,0,4,1,0.0,...,No Deposit,9,NULL,0,Transient,68.0,0,1,Canceled,2015-04-30
3,1,92,2015,July,27,1,2,4,2,0.0,...,No Deposit,9,NULL,0,Transient,76.5,0,2,Canceled,2015-06-23
4,1,100,2015,July,27,2,0,2,2,0.0,...,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-04-02


In [6]:
# Obtener características estadísticas de la tabla con la que estamos trabajando
df.describe()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,BookingChanges,DaysInWaitingList,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests
count,79330.000000,79330.000000,79330.000000,79330.000000,79330.000000,79330.000000,79330.000000,79330.000000,79326.000000,79330.000000,79330.000000,79330.000000,79330.000000,79330.000000,79330.000000,79330.000000,79330.000000,79330.000000
mean,0.417270,109.735724,2016.174285,27.177449,15.786625,0.795185,2.182957,1.850977,0.091370,0.004941,0.025615,0.079743,0.132371,0.187369,3.226774,105.304465,0.024367,0.546918
std,0.493111,110.948526,0.699181,13.398523,8.728451,0.885026,1.456416,0.509292,0.372177,0.084323,0.157983,0.415472,1.693411,0.608620,20.870890,43.602954,0.154919,0.780776
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,23.000000,2016.000000,17.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,79.200000,0.000000,0.000000
50%,0.000000,74.000000,2016.000000,27.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.900000,0.000000,0.000000
75%,1.000000,163.000000,2017.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,126.000000,0.000000,1.000000
max,1.000000,629.000000,2017.000000,53.000000,31.000000,16.000000,41.000000,4.000000,3.000000,10.000000,1.000000,21.000000,72.000000,21.000000,391.000000,5400.000000,3.000000,5.000000


In [7]:
# Filtramos el DataFrame para tener el período muestral debe ser desde la semana 27 del 2015 (julio) a la 35 del 2017 (agosto).

# Convertir la columna 'ArrivalDateWeekNumber' a tipo numérico
df['ArrivalDateWeekNumber'] = pd.to_numeric(df['ArrivalDateWeekNumber'], errors='coerce')

# Filtrar el DataFrame según el período muestral
df_filtrado = df[
    (df['ArrivalDateYear'] == 2015) & (df['ArrivalDateWeekNumber'] >= 27) |
    (df['ArrivalDateYear'] > 2015) & (df['ArrivalDateYear'] < 2017) |
    (df['ArrivalDateYear'] == 2017) & (df['ArrivalDateWeekNumber'] <= 35)
]

# Verificar el nuevo rango de fechas
print(df_filtrado[['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateWeekNumber']])

       ArrivalDateYear ArrivalDateMonth  ArrivalDateWeekNumber
0                 2015             July                     27
1                 2015             July                     27
2                 2015             July                     27
3                 2015             July                     27
4                 2015             July                     27
...                ...              ...                    ...
79325             2017           August                     35
79326             2017           August                     35
79327             2017           August                     35
79328             2017           August                     35
79329             2017           August                     35

[79330 rows x 3 columns]


### Análisis explotarorio 

In [8]:
tasa_cancelacion = df_filtrado['IsCanceled'].mean()
print(f'Tasa de cancelación: {tasa_cancelacion:.2%}')

Tasa de cancelación: 41.73%


In [11]:
# Observamos una tasa de cancelación de 41.73% por lo que hay una tasa de reserva de 58.27%.

In [13]:
# Clientes potenciales (no han cancelado y son repetidos)

# Queremos ver cuales son los gustos de nuestros clientes mas fieles(target)
clientes_potenciales = df_filtrado[(df_filtrado['IsCanceled'] == 0) & (df_filtrado['IsRepeatedGuest'] == 1)]
caracteristicas_clientes_potenciales = clientes_potenciales[['LeadTime', 'MarketSegment', 'Meal', 'PreviousBookingsNotCanceled', 'RequiredCarParkingSpaces']]

# Resumen estadístico de las características de los clientes potenciales
resumen_clientes_potenciales = caracteristicas_clientes_potenciales.describe(include='all')

print('Características de los clientes potenciales:\n', caracteristicas_clientes_potenciales.head())
print('Resumen estadístico de las características de los clientes potenciales:\n', resumen_clientes_potenciales)

Características de los clientes potenciales:
        LeadTime  MarketSegment       Meal  PreviousBookingsNotCanceled  \
34637         0  Complementary  BB                                   1   
34638         1  Complementary  BB                                   2   
34639         0  Complementary  BB                                   3   
34640         1  Complementary  BB                                   4   
34641         0  Complementary  BB                                   5   

       RequiredCarParkingSpaces  
34637                         0  
34638                         0  
34639                         0  
34640                         0  
34641                         0  
Resumen estadístico de las características de los clientes potenciales:
            LeadTime MarketSegment       Meal  PreviousBookingsNotCanceled  \
count   1591.000000          1591       1591                  1591.000000   
unique          NaN             7          4                          NaN   
t

In [14]:
# Resumen de las características de los clientes potenciales
resumen_clientes_potenciales = caracteristicas_clientes_potenciales.describe(include='all')

# Distribución de MarketSegment entre los clientes potenciales
distribucion_segmento_mercado_clientes_potenciales = clientes_potenciales['MarketSegment'].value_counts()

# Distribución de Meal entre los clientes potenciales
distribucion_meal_clientes_potenciales = clientes_potenciales['Meal'].value_counts()

print('\nDistribución de MarketSegment entre los clientes potenciales:\n', distribucion_segmento_mercado_clientes_potenciales)
print('\nDistribución de Meal entre los clientes potenciales:\n', distribucion_meal_clientes_potenciales)


Distribución de MarketSegment entre los clientes potenciales:
 MarketSegment
Corporate        842
Online TA        196
Complementary    193
Direct           158
Offline TA/TO    142
Aviation          53
Groups             7
Name: count, dtype: int64

Distribución de Meal entre los clientes potenciales:
 Meal
BB           1448
SC             98
HB             42
FB              3
Name: count, dtype: int64


In [16]:
# Clientes potenciales que han cancelado
clientes_cancelados = df_filtrado[df_filtrado['IsCanceled'] == 1]
caracteristicas_clientes_cancelados = clientes_cancelados[['LeadTime', 'MarketSegment', 'Meal', 'PreviousBookingsNotCanceled', 'RequiredCarParkingSpaces']]

# Resumen de las características de clientes que han cancelado
resumen_clientes_cancelados = caracteristicas_clientes_cancelados.describe(include='all')

print('Características de los clientes que han cancelado:\n', caracteristicas_clientes_cancelados.head())
print('\nResumen estadístico de las características de clientes que han cancelado:\n', resumen_clientes_cancelados)

Características de los clientes que han cancelado:
    LeadTime MarketSegment       Meal  PreviousBookingsNotCanceled  \
1        88     Online TA  BB                                   0   
2        65     Online TA  BB                                   0   
3        92     Online TA  BB                                   0   
4       100     Online TA  BB                                   0   
5        79     Online TA  BB                                   0   

   RequiredCarParkingSpaces  
1                         0  
2                         0  
3                         0  
4                         0  
5                         0  

Resumen estadístico de las características de clientes que han cancelado:
             LeadTime MarketSegment       Meal  PreviousBookingsNotCanceled  \
count   33102.000000         33102      33102                 33102.000000   
unique           NaN             8          4                          NaN   
top              NaN     Online TA  BB     

In [17]:
# Distribución de MarketSegment entre los clientes que han cancelado
distribucion_segmento_mercado_clientes_cancelados = clientes_cancelados['MarketSegment'].value_counts()

# Distribución de Meal entre los clientes que han cancelado
distribucion_meal_clientes_cancelados = clientes_cancelados['Meal'].value_counts()

print('\n Distribución de MarketSegment entre los clientes que han cancelado:\n', distribucion_segmento_mercado_clientes_cancelados)
print('\n Distribución de Meal entre los clientes que han cancelado:\n', distribucion_meal_clientes_cancelados)


 Distribución de MarketSegment entre los clientes que han cancelado:
 MarketSegment
Online TA        14491
Groups            9623
Offline TA/TO     7173
Direct            1056
Corporate          641
Complementary       64
Aviation            52
Undefined            2
Name: count, dtype: int64

 Distribución de Meal entre los clientes que han cancelado:
 Meal
BB           26667
SC            3963
HB            2437
FB              35
Name: count, dtype: int64


### 1. Elaboración de Series Temporales

In [10]:
# Crear la nueva columna 'Fecha'
df_filtrado['Fecha'] = pd.to_datetime(df_filtrado['ArrivalDateYear'].astype(str) + '-' + df_filtrado['ArrivalDateMonth'] + '-' + df_filtrado['ArrivalDateDayOfMonth'].astype(str), format='%Y-%B-%d')

# El formato '%Y-%B-%d' representa Año-Mes-Día

# Visualizar el DataFrame
print(df_filtrado[['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth', 'Fecha']])

       ArrivalDateYear ArrivalDateMonth  ArrivalDateDayOfMonth      Fecha
0                 2015             July                      1 2015-07-01
1                 2015             July                      1 2015-07-01
2                 2015             July                      1 2015-07-01
3                 2015             July                      1 2015-07-01
4                 2015             July                      2 2015-07-02
...                ...              ...                    ...        ...
79325             2017           August                     30 2017-08-30
79326             2017           August                     31 2017-08-31
79327             2017           August                     31 2017-08-31
79328             2017           August                     31 2017-08-31
79329             2017           August                     29 2017-08-29

[79330 rows x 4 columns]


In [11]:
# Eliminar las columnas que no sirven
df_filtrado = df_filtrado.drop(['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth','Meal', 'Country', 'MarketSegment', 'DistributionChannel','IsRepeatedGuest', 'ReservedRoomType', 'AssignedRoomType',
       'BookingChanges', 'DepositType', 'Agent', 'Company',
       'DaysInWaitingList', 'CustomerType', 'ADR', 'RequiredCarParkingSpaces',
       'TotalOfSpecialRequests'], axis=1)

In [12]:
# Configurar 'Fecha' como el índice
df_filtrado.set_index('Fecha', inplace=True)

In [13]:
df_filtrado.head()

,IsCanceled,LeadTime,ArrivalDateWeekNumber,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,PreviousCancellations,PreviousBookingsNotCanceled,ReservationStatus,ReservationStatusDate
Fecha,,,,,,,,,,,,
2015-07-01,0,6,27,0,2,1,0.0,0,0,0,Check-Out,2015-07-03
2015-07-01,1,88,27,0,4,2,0.0,0,0,0,Canceled,2015-07-01
2015-07-01,1,65,27,0,4,1,0.0,0,0,0,Canceled,2015-04-30
2015-07-01,1,92,27,2,4,2,0.0,0,0,0,Canceled,2015-06-23
2015-07-02,1,100,27,0,2,2,0.0,0,0,0,Canceled,2015-04-02


In [14]:
# Crear columnas adicionales para contar reservas y cancelaciones
df_filtrado['Reservas'] = 1 - df_filtrado['IsCanceled']
df_filtrado['Cancelaciones'] = df_filtrado['IsCanceled']

# Agrupar por semana y realizar la suma
reservas_semanales = df_filtrado.resample('W').sum()[['Reservas']]
cancelaciones_semanales = df_filtrado.resample('W').sum()[['Cancelaciones']]

In [15]:
# Guardar el DataFrame modificado en un nuevo archivo CSV
df_filtrado.to_csv('df_filtrado.csv', index=False)

In [16]:
# Guardar las tablas de reservas y cancelaciones en archivos CSV
reservas_semanales.to_csv('reservas_semanales.csv')
cancelaciones_semanales.to_csv('cancelaciones_semanales.csv')